In [1]:
from Research_Scraper_Code.Research_Scraper import ResearchScraper
import json
import pandas as pd
import time
import json

# Test Notebook for application of scraping
Demo Links

In [3]:
# test urls
url_ieee = 'https://ieeexplore.ieee.org/document/7887648'
url_springer = 'https://link.springer.com/article/10.1007/s12525-020-00445-0'
url_doi = 'https://doi.org/10.1007/978-0-387-73947-2_8'
url_elsevier = 'https://www.sciencedirect.com/science/article/pii/S2451929420300851?via%3Dihub'
url_springer2 = 'https://doi.org/10.1007/s00450-009-0054-z'

Init scraper

In [2]:
scraper = ResearchScraper()

In [3]:
def load_publications_from_csv():
    data = 'data/publications_without_abstract.csv'

    with open(data) as f:
        df = pd.read_csv(f, sep=';')
    return df


df_publications = load_publications_from_csv()

Get entries with DOI

In [4]:
def get_all_dois(df):
    dois = df['doi']
    # remove NaNs
    dois = dois.dropna()
    dois.tolist()
    return dois


publication_dois = get_all_dois(df_publications)

Method for creating JSON file as output

In [5]:
def write_results(results, name):
    if results is not None:
        with open(f'exports/scrapings/{name}.json', 'w') as f:
            json.dump(results, f, indent=4)

Create a sample of cris data and scrape them al

In [6]:
def scrape_sample_of_dois(dois, n):
    # get a sample of 10 from dois
    sample = dois.sample(n)
    #print(sample)
    results = []

    for doi in sample:
        print(f'Scraping {doi}')
        start = time.time()
        result = scraper.scrape_publication_by_doi(doi, params=['full'])
        end = time.time()
        print(f'Total time : {end - start}')
        old_len = len(results)
        results.append(result)
        print(f'\n \t  >>>>>> added new result, n went from {old_len} to n={len(results)}')
        # print(f'\t -> Results: {results}')

    #write_results(results, f'sample_{time.strftime("%Y_%m_%d__%H_%M")}')
    return results


In [7]:
# timestamp with date and time
timestamp = time.strftime("%Y_%m_%d__%H_%M")
timestamp

'2022_10_16__13_23'

In [8]:
# scrape sample of 10
res = None
res = scrape_sample_of_dois(publication_dois, 3)
write_results(res, f'sample_{time.strftime("%Y_%m_%d__%H_%M")}')



Scraping 10.1111/1467-8551.12350
URL ('https://doi.org/10.1111/1467-8551.12350') is a DOI link, Links is now resolved properly
Resolved DOI link to: https://onlinelibrary.wiley.com/doi/10.1111/1467-8551.12350
[DEBUG - ResearchScraper] - No scraper found for https://onlinelibrary.wiley.com/doi/10.1111/1467-8551.12350
Total time : 0.6439321041107178

 	  >>>>>> added new result, n went from 0 to n=1
Scraping 10.1016/j.eneco.2016.03.008
URL ('https://doi.org/10.1016/j.eneco.2016.03.008') is a DOI link, Links is now resolved properly
Resolved DOI link to: https://linkinghub.elsevier.com/retrieve/pii/S0140988316300469
[DEBUG - ResearchScraper] - Found scraper for https://linkinghub.elsevier.com/retrieve/pii/S0140988316300469 -> ScraperScienceDirect
200


KeyboardInterrupt: 

Scraping a list of DOIs

In [9]:
def scrape_publication_by_doi_list(doi_list, params=['full']):
    print(f'Time of scrape start: {time.strftime("%Y_%m_%d__%H_%M")}')
    results = []
    for doi in doi_list:
        print(f'>>> Scraping {doi}')
        result = scraper.scrape_publication_by_doi(doi, params)
        print(f'>>>> Scraping {doi} done')
        results.append(result)
        print(f'>>>> Scraping {doi} added to results')
    print(f'>>>> Scraping {len(doi_list)} publications done')
    write_results(results, f'scrapings_{time.strftime("%Y_%m_%d__%H_%M")}')
    print(f'Time of scrape end: {time.strftime("%Y_%m_%d__%H_%M")}')
    return results

In [17]:
# Draw a sample list and scrape them
doi_list_sample = publication_dois.sample(10)
scraping_results = scrape_publication_by_doi_list(doi_list_sample)
len(scraping_results)

>>> Scraping 10.1509/jmkg.72.2.1
URL ('https://doi.org/10.1509/jmkg.72.2.1') is a DOI link, Links is now resolved properly
Resolved DOI link to: https://journals.sagepub.com/doi/10.1509/jmkg.72.2.1
[DEBUG - ResearchScraper] - No scraper found for https://journals.sagepub.com/doi/10.1509/jmkg.72.2.1
>>>> Scraping 10.1509/jmkg.72.2.1 done
>>>> Scraping 10.1509/jmkg.72.2.1 added to results
>>>> Scraping 1 publications done


1

In [ ]:
# Scrape all dois
scraping_results_all_dois = scrape_publication_by_doi_list(publication_dois)
print(len(scraping_results_all_dois))

## Importing results of scraping

In [10]:
# read json file with scraping results (all cris doi numbers)
with open('exports/scrapings/scrapings_2022_10_16__07_09.json') as f:
    scraping_results_imported = json.load(f)
# old file 582 after clean

In [11]:
# drop Nones from dict
scraping_results_imported_cleaned = [x for x in scraping_results_imported if x is not None]
# old file 582 after clean

In [12]:
# get the rows from the dict with the key 'error'
error_rows = [x for x in scraping_results_imported_cleaned if x.get('error') is not None]

In [22]:
# print time
print(f'Time: {time.strftime("%Y_%m_%d__%H_%M")}')

Time: 2022_10_16__00_48
